In [1]:
import pandas as pd
import sklearn
from sklearn.feature_extraction.text import CountVectorizer
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.layers import Dense,Dropout,Activation
from tensorflow.keras.layers import Conv1D ,GlobalMaxPooling1D
from tensorflow.keras.layers import Embedding
from tensorflow.keras.datasets import imdb
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

In [2]:
# 绘制模型表现图
def performance(y_true , predict , color = "g" , ann = True):
    acc = accuracy_score(y_true , predict[:] > 0.5)
    auc = roc_auc_score(y_true , predict[:])
    fpr , tpr , thr = roc_curve(y_true , predict[:])
    plt.figure()
    plt.plot(fpr , tpr )

# 读取数据

In [3]:
df_train = pd.read_csv("training.csv",sep=',',header=0,encoding='utf-8')
df_validation = pd.read_csv("validation.csv",sep=',',header=0,encoding='utf-8')
# print(df_validation.head(50))
train_X = df_train['content']
train_Y = df_train['label']
validate_X = df_validation['content']
validate_Y = df_validation['label']
train_Y = tf.squeeze(train_Y)
validate_Y = tf.squeeze(validate_Y)

# 预处理

In [4]:
train_X.shape,train_Y.shape,validate_X.shape,validate_Y.shape

((5838,), TensorShape([5838]), (730,), TensorShape([730]))

In [5]:
import jieba
import string
def clean_CN(corpus):
    stop = []
    with open(r"D:\大创项目\LDA\stopwords\CNstopwords.txt", 'r', encoding='utf-8') as f:
        for lines in f:
            stop.append(lines.strip())
    stop = set(stop)
   
    exclude = set(string.punctuation)  # 标点符号
    clean_corpus = []
    for doc in corpus:
        words = jieba.lcut(doc)
        stop_free = [i for i in words if (i not in stop) & (i.isalpha())]
        clean_corpus.append(stop_free)
    return clean_corpus


In [6]:
train_X = clean_CN(train_X)
validate_X = clean_CN(validate_X)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\Pluto\AppData\Local\Temp\jieba.cache
Loading model cost 1.472 seconds.
Prefix dict has been built succesfully.


In [8]:
train_Y

<tf.Tensor: id=1, shape=(5838,), dtype=int32, numpy=array([2, 2, 2, ..., 1, 3, 3])>

# 特征提取 

In [9]:
from gensim import corpora
texts = train_X.copy()
texts.extend(validate_X)
print(len(texts))
print(len(train_X))
print(len(validate_X))

6568
5838
730


In [10]:
dictionary = corpora.Dictionary(texts)
dictionary.filter_extremes(no_below=10, no_above=0.5)
dictionary.compactify() 
print(dictionary)

Dictionary(2358 unique tokens: ['一回', '乙', '书', '同学', '对不起']...)


In [11]:
train_X = [dictionary.doc2bow(text) for text in train_X]
validate_X = [dictionary.doc2bow(text) for text in validate_X]


In [12]:
new_train_X = []
new_validate_X = []
for i in range(len(train_X)):
    temp = []
    doc = train_X[i]
    for k in range(len(doc)):
        count = doc[k][1]
        for j in range(count):
            temp.append(doc[k][0])
    new_train_X.append(temp)
print(len(new_train_X))
print(new_train_X[10])

for i in range(len(validate_X)):
    temp = []
    doc = validate_X[i]
    for k in range(len(doc)):
        count = doc[k][1]
        for j in range(count):
            temp.append(doc[k][0])
    new_validate_X.append(temp)
print(len(new_validate_X))
print(new_validate_X[10])

5838
[14, 25, 38, 47, 95, 95, 108, 109, 110, 111, 112, 113, 113, 114, 115, 116, 117, 118, 119]
730
[18, 18, 18, 211, 256, 257, 320, 322, 412, 452, 790, 1025, 1190, 1392, 1631, 1886, 1886, 1886]


In [13]:
from tensorflow.keras.preprocessing import sequence
train_x = sequence.pad_sequences(new_train_X, maxlen=15)
validate_x = sequence.pad_sequences(new_validate_X, maxlen=15)

In [14]:
train_x.shape,validate_x.shape

((5838, 15), (730, 15))

# 构建网络

In [15]:
max_features = 2358
embedding_dims = 15
maxlen = 15
filters =250
kernel_size = 3
hidden_dims = 250
batch_size = 128
epochs = 100


model = keras.Sequential(
    [
        keras.layers.Embedding(max_features,
                    embedding_dims,
                    input_length=maxlen),
        keras.layers.Dropout(0.2),
        
        keras.layers.Conv1D(filters,
                 kernel_size,
                 padding='same',
                 activation='relu',
                 strides=1,
                ),
        
        keras.layers.Activation('relu'),
        keras.layers.MaxPooling1D(),
        
        
        keras.layers.Conv1D(filters,
                 kernel_size,
                 padding='same',
                 activation='relu',
                 strides=1),
        keras.layers.Activation('relu'),
        keras.layers.MaxPooling1D(),
        
        keras.layers.Dropout(0.2),
        
                
        keras.layers.Conv1D(filters,
                 kernel_size,
                 padding='same',
                 activation='relu',
                 strides=1),
        keras.layers.Activation('relu'),
        keras.layers.MaxPooling1D(),
        
        
        keras.layers.Flatten(),
        keras.layers.Dense(hidden_dims),
        keras.layers.Dropout(0.2),
        keras.layers.Activation('relu'),
        keras.layers.Dense(1),
        keras.layers.Activation('sigmoid'),
        
    ]

)
opt = keras.optimizers.Adam(learning_rate=0.01)
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])
model.summary()

model.fit(train_x, train_Y,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(validate_x, validate_Y))


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 15, 15)            35370     
_________________________________________________________________
dropout (Dropout)            (None, 15, 15)            0         
_________________________________________________________________
conv1d (Conv1D)              (None, 15, 250)           11500     
_________________________________________________________________
activation (Activation)      (None, 15, 250)           0         
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 7, 250)            0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 7, 250)            187750    
_________________________________________________________________
activation_1 (Activation)    (None, 7, 250)            0

5838/5838 [==============================] - 3s 565us/sample - loss: 3.0211e-07 - accuracy: 0.1146 - val_loss: 3.0243e-07 - val_accuracy: 0.1274
Epoch 34/100
5838/5838 [==============================] - 3s 570us/sample - loss: 3.0211e-07 - accuracy: 0.1146 - val_loss: 3.0243e-07 - val_accuracy: 0.1274
Epoch 35/100
5838/5838 [==============================] - 3s 563us/sample - loss: 3.0211e-07 - accuracy: 0.1166 - val_loss: 3.0243e-07 - val_accuracy: 0.1274
Epoch 36/100
5838/5838 [==============================] - 3s 558us/sample - loss: 3.0211e-07 - accuracy: 0.1136 - val_loss: 3.0243e-07 - val_accuracy: 0.1274
Epoch 37/100
5838/5838 [==============================] - 3s 553us/sample - loss: 3.0211e-07 - accuracy: 0.1155 - val_loss: 3.0243e-07 - val_accuracy: 0.1274
Epoch 38/100
5838/5838 [==============================] - 3s 552us/sample - loss: 3.0211e-07 - accuracy: 0.1175 - val_loss: 3.0243e-07 - val_accuracy: 0.1274
Epoch 39/100
5838/5838 [==============================] - 3s 566u

Epoch 85/100
5838/5838 [==============================] - 4s 710us/sample - loss: 3.0211e-07 - accuracy: 0.1143 - val_loss: 3.0243e-07 - val_accuracy: 0.1274
Epoch 86/100
5838/5838 [==============================] - 4s 700us/sample - loss: 3.0211e-07 - accuracy: 0.1185 - val_loss: 3.0243e-07 - val_accuracy: 0.1274
Epoch 87/100
5838/5838 [==============================] - 4s 628us/sample - loss: 3.0211e-07 - accuracy: 0.1155 - val_loss: 3.0243e-07 - val_accuracy: 0.1274
Epoch 88/100
5838/5838 [==============================] - 4s 667us/sample - loss: 3.0211e-07 - accuracy: 0.1178 - val_loss: 3.0243e-07 - val_accuracy: 0.1274
Epoch 89/100
5838/5838 [==============================] - 4s 666us/sample - loss: 3.0211e-07 - accuracy: 0.1155 - val_loss: 3.0243e-07 - val_accuracy: 0.1274
Epoch 90/100
5838/5838 [==============================] - 4s 637us/sample - loss: 3.0211e-07 - accuracy: 0.1170 - val_loss: 3.0243e-07 - val_accuracy: 0.1274
Epoch 91/100
5838/5838 [============================